In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))
sys.path.append(os.path.join(os.getcwd(), "molgen"))
sys.path.append(os.path.join(os.getcwd(), "iclr19-graph2graph/props"))


%load_ext autoreload
%autoreload 2

from otgnn.models import GCN
# from molgen.dataloading.MolGraphBatchPreprocessor import MolGraph
from otgnn.graph import MolGraph
from otgnn.utils import save_model, load_model
from otgnn.graph import SYMBOLS, FORMAL_CHARGES, BOND_TYPES


from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args
from mol_opt.train_mol_opt import main, load_checkpoint, initialize_models
from mol_opt.mol_opt import MolOpt
from mol_opt.decoder_mol_opt import MolOptDecoder
from mol_opt.ot_utils import encode_target
from mol_opt.ot_utils import FGW 
from mol_opt.ot_utils import compute_barycenter

from molgen.dataloading.feat2smiles import feat2smiles
from molgen.dataloading.mol_drawer import MolDrawer
from molgen.metrics.Penalty import Penalty, RecPenalty
from molgen.metrics.mol_metrics import MolMetrics

from rdkit.Chem import MolFromSmiles
from tensorboardX import SummaryWriter

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import grad
import numpy as np
import time
from rdkit import Chem
from rdkit.Chem.Draw import IPythonConsole
from molgen.metrics.Penalty import Penalty as PenaltyNew

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')

/home/octav/gitrepos/tum-thesis


In [31]:
np.random.randint(2)

1

In [40]:
for i in range(3):
    print (i)

0
1
2


In [2]:
model_type = "deepsets"
sys.argv = ["", "-model_type", model_type, "-one_batch_train"]
args = get_args()

molopt = MolOpt(args)
molopt_decoder = MolOptDecoder(args)

In [3]:
# train_data_loader = get_loader("iclr19-graph2graph/data/qed", "train_pairs", args.batch_size, True)
datatype = "train_split"
train_data_loader = get_loader("molgen/data/chembl50", datatype, 50, same_number_atoms = True)

for i in train_data_loader:
#     X = (MolGraph(i[0]))
#     Y = (MolGraph(i[1]))
    X = MolGraph(i)
    Y = X
    break

In [7]:
x_embedding = molopt.encode(X)

In [5]:
from mol_opt.transformer import make_model
from torch.autograd import Variable
model = make_model(args)
# model.forward(x_embedding, x_embedding).shape

In [6]:
model(x_embedding, x_embedding, None, None)

RuntimeError: The size of tensor a (150) must match the size of tensor b (300) at non-singleton dimension 1

In [21]:
i = 5
x_embedding[i].shape
ys = torch.zeros((1,args.pc_hidden))

for i in range(X.scope[i][1]):
    print (i, ys.shape)
    out = model(x_embedding[i], Variable(ys), None, None)
    ys = torch.cat([ys, out[-1].unsqueeze(0)])
#     ys = torch.cat([ys, 
#                     torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)

IndexError: tuple index out of range

In [23]:
x_embedding

AttributeError: 'tuple' object has no attribute 'shape'

In [10]:
out.shape

torch.Size([23, 150])

In [11]:
x_embedding[i].shape

torch.Size([150])

In [12]:
x_embedding.shape[]

SyntaxError: invalid syntax (<ipython-input-12-e4dd6f5b256a>, line 1)

In [9]:
x_encoding, x_embedding = molopt.forward(X)
yhat_logits = molopt_decoder.forward(x_embedding, X, Y)
yhat_labels = molopt_decoder.discretize_argmax(*yhat_logits)
# yhat_labels = molopt_decoder.discretize_gumbel(*yhat_logits, tau = pen.tau)
pred_pack = (yhat_labels, yhat_logits, Y.scope), Y 

target = Y.get_graph_outputs()
symbols_labels, charges_labels, bonds_labels = yhat_labels
symbols_logits, charges_logits, bonds_logits = yhat_logits

pen.mirror = ""

pen.tau = 0.05
fgw_loss_item = fgw_loss(*pred_pack, tau = 1)
print (fgw_loss_item.item()/args.batch_size)
pen_loss = pen(*pred_pack, model_iter)
print (pen_loss)
metrics.measure_batch(pred_pack[0], target)

NameError: name 'pen' is not defined

In [13]:
x_encoding.shape

torch.Size([1167, 300])

In [14]:
ys.shape

torch.Size([24, 150])

In [31]:
memory = model.encode(x_embedding)

In [40]:
model(memory, x_embedding)

NameError: name 'memory' is not defined

In [104]:
max_len = 5 
ys = torch.zeros(50, args.pc_hidden)
for i in range(max_len-1):
    out = model.decode(memory,
                       Variable(ys)) 
    ys = torch.cat((ys, out)).view(50,2,150)

RuntimeError: The size of tensor a (2) must match the size of tensor b (100) at non-singleton dimension 1

In [105]:
out.shape

torch.Size([50, 150])

In [106]:
ys.shape

torch.Size([50, 2, 150])

In [53]:
torch.from_numpy(np.random.randint(1, 20, size=(50, 10))).shape

torch.Size([50, 10])

In [34]:
x_encoding, Z = molopt.forward(X)

AttributeError: 'NoneType' object has no attribute 'mean'

In [70]:
molopt.transformer(Z[0], None).shape

torch.Size([1, 150])

In [52]:
Z.unsqueeze(1).shape

torch.Size([50, 1, 150])

In [73]:
Z.unsqueeze(1).repeat(1,5,1).shape

torch.Size([50, 5, 150])

In [117]:
molopt.transformer(x_embedding[1], None).shape

torch.Size([1, 150])

In [120]:
x_embedding[1].shape

torch.Size([150])